In [7]:
PORT = 5000
env = 'ICL' # LOCAL | ICL

In [8]:
from pathlib import Path

if env == 'LOCAL':
    WORK_DIR = Path('.')
elif env == 'ICL':
    import os, sys
    WORK_DIR = Path('/vol/bitbucket/') / os.getenv("USER") / 'worker'
    # !pip install -r {WORK_DIR}/requirements.txt

    if not WORK_DIR in sys.path:
        sys.path.append(str(WORK_DIR))


os.environ['WORK_DIR'] = str(WORK_DIR)

In [9]:
import uvicorn
import nest_asyncio
from fastapi import FastAPI, WebSocket, BackgroundTasks
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
from src.constants import UI_URL, WORKER_URL
from src.models.requests import WorkUnit, Request
from src.socket_service import SocketService
from src.work import process_work
from dummy_response import audit
import logging

nest_asyncio.apply()

app = FastAPI()

origins = [
    str(UI_URL),
    str(WORKER_URL),
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post("/process")
async def process(request: Request, background_tasks: BackgroundTasks):
    SocketService.register_incoming_id(request.socket)
    background_tasks.add_task(process_work, request.work)
    return JSONResponse(content={"status": 200})

@app.websocket("/ws/{socket_id}")
async def websocket_endpoint(websocket: WebSocket, socket_id: str):
    async with SocketService(websocket, socket_id) as connection:
        await connection.listen()


if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=PORT)

INFO:     Started server process [3955976]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:5000 (Press CTRL+C to quit)


INFO:     127.0.0.1:52808 - "POST /process HTTP/1.1" 422 Unprocessable Entity


INFO:     ('127.0.0.1', 52816) - "WebSocket /ws/8438e6fb-4db7-46a3-bb82-8aa5b82d4d85" 403
INFO:     connection rejected (403 Forbidden)
INFO:     connection closed
